In [5]:
from datetime import datetime
import datetime as dt
import time
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import pandas as pd
import duckdb as db
import seaborn as sns
import matplotlib.pyplot as plt
import IPython as ip
import  os as os


****************************************************************************************************************
****************************************************************************************************************
AS ALWAYS START WITH SILO
https://app.silo.finance/silo/0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db?screen=deposit

In [6]:
sg = Subgrounds()
apiKey = input('API Key, please')

In [79]:
#for when we save the files
file = input('Selet a folder to save output') ##enter your file path here - the file is in the repo "summary_stats.csv".
file = file+'/data'
if not os.path.exists(file):
    os.makedirs(file)

In [80]:
#SILO subgraph: https://gateway.thegraph.com/api/[api-key]/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB
silo = sg.load_api('https://gateway.thegraph.com/api/'+apiKey+'/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB')

In [410]:
oldBorrowsDb = pd.DataFrame()
siloFileName = file+'/siloFinanceOhmBorrows.csv'
try:
    oldBorrowsDb = pd.read_csv (siloFileName)
except:
    oldBorrowsDb = pd.DataFrame()
oldBorrowsDb.head(100)

""


In [411]:
maxSiloBorrowTimestamp = 0
try:
    maxSiloBorrowTimestamp = (max(0,oldBorrowsDb['transaction_timestamp'].max()))-5
except:
        maxSiloBorrowTimestamp = 0
maxSiloBorrowTimestamp = int(maxSiloBorrowTimestamp)
maxSiloBorrowTimestamp

0

In [412]:
#0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db

In [413]:
##this captures the ENTIRE list of people who borrowed OHM
borrowTicker = 0
borrowslist = pd.DataFrame()
siloBorrowsDb = pd.DataFrame()
borrowListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (borrowTicker)*(1000)
    borrows = silo.Query.borrows(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db'
                ,'timestamp_gte': maxSiloBorrowTimestamp
                } #gOHM market
    )

    borrowslist = sg.query_df([
        borrows.id
        ,borrows.hash
        ,borrows.blockNumber
        ,borrows.timestamp
        ,borrows.account.id
        ,borrows.market.id
        ,borrows.market.protocol.id
        ,borrows.market.rates.rate
        ,borrows.market.rates.duration
        ,borrows.market.rates.side
        ,borrows.market.rates.type
        ,borrows.market.rates.token.id
        ,borrows.asset.name
        ,borrows.asset.symbol
        ,borrows.asset.decimals
        ,borrows.asset.lastPriceUSD
        ,borrows.amount
        ,borrows.amountUSD
    ])

    siloBorrowsDb=pd.concat([siloBorrowsDb, borrowslist])
    recordTimestamp1 = siloBorrowsDb.iat[borrowTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    siloBorrowsDbLength = len(siloBorrowsDb)
    borrowListLength = len(borrowslist)
    recordID = siloBorrowsDb.iat[borrowTicker,0]
    borrow = siloBorrowsDb.iat[borrowTicker,1]
    print("iterations: ", borrowTicker, "Lines skipped: ",skipValue, "records collected: ", borrowListLength, " - latest recordID: ",borrow, " ", recordID , " - borrows DB length: ", siloBorrowsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", borrowTicker, "records collected: ", borrowListLength, " - userdb length: ", borrowsDbLength)
    borrowTicker = borrowTicker+1

    if daysAgo>daysLimit: exit =True
    if borrowListLength<1000: exit =True

siloBorrowsDb.columns = siloBorrowsDb.columns.str.replace("borrows", "transaction")
siloBorrowsDb['event']='borrow'
siloBorrowsDb.drop_duplicates
print("Done. Total Records collected: ", siloBorrowsDbLength)

iterations:  0 Lines skipped:  0 records collected:  24  - latest recordID:  0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e611a9264bcb3d4f9dd6ae8   0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e611a9264bcb3d4f9dd6ae8-135  - borrows DB length:  24  - latest record from:  2023-02-17 04:21:59 2 days ago
Done. Total Records collected:  24


In [414]:
siloBorrowsDb.head(100)

,transaction_id,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_market_protocol_id,transaction_market_rates_rate,transaction_market_rates_duration,transaction_market_rates_side,transaction_market_rates_type,transaction_market_rates_token_id,transaction_asset_name,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,transaction_amountUSD,event
0,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,16648400,1676636519,0xcb6e1613029d790c00f89296808f278d6dc25b2f,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,1.164533,None,BORROWER,VARIABLE,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,Olympus,OHM,9,10.668690,58000000000,615.628222,borrow
1,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,16648400,1676636519,0xcb6e1613029d790c00f89296808f278d6dc25b2f,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Olympus,OHM,9,10.668690,58000000000,615.628222,borrow
2,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,16648400,1676636519,0xcb6e1613029d790c00f89296808f278d6dc25b2f,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Olympus,OHM,9,10.668690,58000000000,615.628222,borrow
3,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,16648400,1676636519,0xcb6e1613029d790c00f89296808f278d6dc25b2f,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.096868,None,LENDER,VARIABLE,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,Olympus,OHM,9,10.668690,58000000000,615.628222,borrow
4,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,16648400,1676636519,0xcb6e1613029d790c00f89296808f278d6dc25b2f,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,LENDER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Olympus,OHM,9,10.668690,58000000000,615.628222,borrow
5,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,16648400,1676636519,0xcb6e1613029d790c00f89296808f278d6dc25b2f,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,LENDER,VARIABLE,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Olympus,OHM,9,10.668690,58000000000,615.628222,borrow
6,0x6726ea9fa581ab0a5fa26b8a112c95e690500f446c39...,0x6726ea9fa581ab0a5fa26b8a112c95e690500f446c39...,16628130,1676391467,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,1.164533,None,BORROWER,VARIABLE,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,Olympus,OHM,9,10.668690,1480000000000,15782.938441,borrow
7,0x6726ea9fa581ab0a5fa26b8a112c95e690500f446c39...,0x6726ea9fa581ab0a5fa26b8a112c95e690500f446c39...,16628130,1676391467,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Olympus,OHM,9,10.668690,1480000000000,15782.938441,borrow
8,0x6726ea9fa581ab0a5fa26b8a112c95e690500f446c39...,0x6726ea9fa581ab0a5fa26b8a112c95e690500f446c39...,16628130,1676391467,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Olympus,OHM,9,10.668690,1480000000000,15782.938441,borrow
9,0x6726ea9fa581ab0a5fa26b8a112c95e690500f446c39...,0x6726ea9fa581ab0a5fa26b8a112c95e690500f446c39...,166281

In [415]:
siloBorrowsDb['transactionReadableAmount'] = siloBorrowsDb['transaction_amount']/(10**(siloBorrowsDb['transaction_asset_decimals']))
siloBorrowsDb['transactionReadableAmount']=siloBorrowsDb['transactionReadableAmount'].astype('float')
siloBorrowsDb['dateTime'] =(pd.to_datetime(siloBorrowsDb['transaction_timestamp'],unit='s'))



#we only want borrowing in GOHM from those that borrowed, so drop anyone that is not on that side
#we only want withdrawals in GOHM from those that borrowed, so drop anyone that is not on that side
siloBorrowsDb.drop(siloWithdrawsDb.loc[(siloBorrowsDb['transaction_market_rates_side']!='BORROWER') &(siloBorrowsDb['transaction_asset_symbol']!='OHM') ].index, inplace=True)
#siloBorrowsDb.drop(siloWithdrawsDb.loc[(siloBorrowsDb['transaction_asset_name']=='Olympus')].index, inplace=True)
siloBorrowsDb = db.query("select distinct * from siloBorrowsDb where lower(transaction_asset_symbol) = 'ohm'").df()

siloBorrowEventsDbTimedClean = siloBorrowsDb[['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime']].copy()
siloBorrowEventsDbTimedClean.sort_values(by=['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime'])
siloBorrowEventsDbTimedClean.drop_duplicates(subset=['transaction_hash'],keep='first', inplace=True)
siloBorrowEventsDbTimedClean['calDate'] = siloBorrowEventsDbTimedClean['dateTime'].dt.normalize()
siloBorrowEventsDbTimedClean.head(100)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,borrow,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,16648400,1676636519,0xcb6e1613029d790c00f89296808f278d6dc25b2f,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,58.000000,10.66869,615.628222,2023-02-17 12:21:59,2023-02-17
6,borrow,0x6726ea9fa581ab0a5fa26b8a112c95e690500f446c39...,16628130,1676391467,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,1480.000000,10.66869,15782.938441,2023-02-14 16:17:47,2023-02-14
12,borrow,0x5fca09aaf64619a5d644ed392a64f5bd08b9e6b4b46e...,16627404,1676382743,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,175.320428,10.66869,1812.167469,2023-02-14 13:52:23,2023-02-14


In [416]:
path =file+'/siloFinanceGohmBorrows.csv'
siloBorrowEventsDbTimedClean.to_csv(path, index = False)

path =file+'/siloFinanceGohmBorrows_piped.csv'
siloBorrowEventsDbTimedClean.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(siloBorrowEventsDbTimedClean),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\ohmBorrowing/data/siloFinanceGohmBorrows_piped.csv file saved, 3  records


In [417]:
totalBorrowedOHM = sum(siloBorrowEventsDbTimedClean['transactionReadableAmount'])
totalBorrowedUSD = sum(siloBorrowEventsDbTimedClean['transaction_amountUSD'])
print('A total of ',totalBorrowedOHM, 'OHM has been borrowed, or USD$',round(totalBorrowedUSD,2),  'over time')

A total of  1713.320427647 OHM has been borrowed, or USD$ 18210.73 over time


************************************************************************************
*************************************************************************************
SILO DEPOSITS

In [418]:
oldSiloDepositsDb = pd.DataFrame()
siloFileName = file+'/siloFinanceGohmDeposits.csv'
try:
    oldSiloDepositsDb = pd.read_csv (siloFileName)
except:
    oldSiloDepositsDb = pd.DataFrame()
oldSiloDepositsDb.head(100)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,deposit,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,16646360,1676611691,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,10.000000,10.66869,105.925340,2023-02-17 05:28:11,2023-02-17
1,deposit,0xf9bbcc923182fb6406e97fce0f92c22c87a284d55812...,16627144,1676379599,0x245cc372c84b3645bf0ffe6538620b04a217988b,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,20000.000000,10.66869,206726.334541,2023-02-14 12:59:59,2023-02-14
2,deposit,0x0aa351eb8ff1ca2bd6c4e65229f83c7e5b596d72c889...,16625541,1676360171,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,2.514698,10.66869,26.317329,2023-02-14 07:36:11,2023-02-14


In [419]:
maxSiloDepositsTimestamp = 0
try:
    maxSiloDepositsTimestamp = (max(0,oldSiloDepositssDb['transaction_timestamp'].max()))-5
except:
        maxSiloDepositsTimestamp = 0
maxSiloDepositsTimestamp = int(maxSiloDepositsTimestamp)
maxSiloDepositsTimestamp

0

In [420]:
##this captures the ENTIRE list of people who deposited OHM
depositTicker = 0
depositslist = pd.DataFrame()
siloDepositsDb = pd.DataFrame()
depositListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (depositTicker)*(1000)
    deposits = silo.Query.deposits(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db'
                ,'timestamp_gte': maxSiloDepositsTimestamp
                } #gOHM market
    )

    depositslist = sg.query_df([
        deposits.id
        ,deposits.hash
        ,deposits.blockNumber
        ,deposits.timestamp
        ,deposits.account.id
        ,deposits.market.id
        ,deposits.market.protocol.id
        ,deposits.market.rates.rate
        ,deposits.market.rates.duration
        ,deposits.market.rates.side
        ,deposits.market.rates.type
        ,deposits.market.rates.token.id
        ,deposits.asset.name
        ,deposits.asset.symbol
        ,deposits.asset.decimals
        ,deposits.asset.lastPriceUSD
        ,deposits.amount
        ,deposits.amountUSD
    ])

    siloDepositsDb=pd.concat([siloDepositsDb, depositslist])
    recordTimestamp1 = siloDepositsDb.iat[depositTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    siloDepositsDbLength = len(siloDepositsDb)
    siloDepositListLength = len(depositslist)
    recordID = siloDepositsDb.iat[depositTicker,0]
    deposit = siloDepositsDb.iat[depositTicker,1]
    print("iterations: ", depositTicker, "Lines skipped: ",skipValue, "records collected: ", siloDepositListLength, " - latest recordID: ",deposit, " ", recordID , " - deposits DB length: ", siloDepositsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", depositTicker, "records collected: ", depositListLength, " - userdb length: ", depositsDbLength)
    depositTicker = depositTicker+1

    if daysAgo>daysLimit: exit =True
    if siloDepositListLength<1000: exit =True

siloDepositsDb.columns = siloDepositsDb.columns.str.replace("deposits", "transaction")
siloDepositsDb['event']='deposit'
siloDepositsDb.drop_duplicates
print("Done. Total Records collected: ", siloDepositsDbLength)

iterations:  0 Lines skipped:  0 records collected:  66  - latest recordID:  0xa3c1b116f0bdff039f4969bd23a7fb83b8ddb2f5efac06cb4880933961810e22   0xa3c1b116f0bdff039f4969bd23a7fb83b8ddb2f5efac06cb4880933961810e22-196  - deposits DB length:  66  - latest record from:  2023-02-17 04:20:47 2 days ago
Done. Total Records collected:  66


In [421]:
siloDepositsDb.head(10)

,transaction_id,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_market_protocol_id,transaction_market_rates_rate,transaction_market_rates_duration,transaction_market_rates_side,transaction_market_rates_type,transaction_market_rates_token_id,transaction_asset_name,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,transaction_amountUSD,event
0,0xa3c1b116f0bdff039f4969bd23a7fb83b8ddb2f5efac...,0xa3c1b116f0bdff039f4969bd23a7fb83b8ddb2f5efac...,16648394,1676636447,0xcb6e1613029d790c00f89296808f278d6dc25b2f,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,1.164533,None,BORROWER,VARIABLE,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,Wrapped Ether,WETH,18,1674.366016,500000000000000000,829.05000,deposit
1,0xa3c1b116f0bdff039f4969bd23a7fb83b8ddb2f5efac...,0xa3c1b116f0bdff039f4969bd23a7fb83b8ddb2f5efac...,16648394,1676636447,0xcb6e1613029d790c00f89296808f278d6dc25b2f,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,1674.366016,500000000000000000,829.05000,deposit
2,0xa3c1b116f0bdff039f4969bd23a7fb83b8ddb2f5efac...,0xa3c1b116f0bdff039f4969bd23a7fb83b8ddb2f5efac...,16648394,1676636447,0xcb6e1613029d790c00f89296808f278d6dc25b2f,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Wrapped Ether,WETH,18,1674.366016,500000000000000000,829.05000,deposit
3,0xa3c1b116f0bdff039f4969bd23a7fb83b8ddb2f5efac...,0xa3c1b116f0bdff039f4969bd23a7fb83b8ddb2f5efac...,16648394,1676636447,0xcb6e1613029d790c00f89296808f278d6dc25b2f,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.096868,None,LENDER,VARIABLE,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,Wrapped Ether,WETH,18,1674.366016,500000000000000000,829.05000,deposit
4,0xa3c1b116f0bdff039f4969bd23a7fb83b8ddb2f5efac...,0xa3c1b116f0bdff039f4969bd23a7fb83b8ddb2f5efac...,16648394,1676636447,0xcb6e1613029d790c00f89296808f278d6dc25b2f,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,LENDER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,1674.366016,500000000000000000,829.05000,deposit
5,0xa3c1b116f0bdff039f4969bd23a7fb83b8ddb2f5efac...,0xa3c1b116f0bdff039f4969bd23a7fb83b8ddb2f5efac...,16648394,1676636447,0xcb6e1613029d790c00f89296808f278d6dc25b2f,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,LENDER,VARIABLE,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Wrapped Ether,WETH,18,1674.366016,500000000000000000,829.05000,deposit
6,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,16646360,1676611691,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,1.164533,None,BORROWER,VARIABLE,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,Olympus,OHM,9,10.668690,10000000000,105.92534,deposit
7,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,16646360,1676611691,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Olympus,OHM,9,10.668690,10000000000,105.92534,deposit
8,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,16646360,1676611691,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Olympus,OHM,9,10.668690,10000000000,105.92534,deposit
9,0x1c37394aaabc24

In [422]:
siloDepositsDb['transactionReadableAmount'] = siloDepositsDb['transaction_amount']/(10**(siloDepositsDb['transaction_asset_decimals']))
siloDepositsDb['transactionReadableAmount']=siloDepositsDb['transactionReadableAmount'].astype('float')
siloDepositsDb['dateTime'] =(pd.to_datetime(siloDepositsDb['transaction_timestamp'],unit='s'))
siloDepositsDb.head(100)

#we only want deposits in GOHM from those that borrowed, so drop anyone that is not on that side
siloDepositsDb.drop(siloDepositsDb.loc[siloDepositsDb['transaction_market_rates_side']!='BORROWER'].index, inplace=True)
siloDepositsDb.drop(siloDepositsDb.loc[siloDepositsDb['transaction_asset_symbol']!='OHM'].index, inplace=True)


siloDepositEventsDbTimedClean = siloDepositsDb[['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime']].copy()
siloDepositEventsDbTimedClean.sort_values(by=['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime'])
siloDepositEventsDbTimedClean.drop_duplicates(subset=['transaction_hash'],keep='first', inplace=True)
siloDepositEventsDbTimedClean['calDate'] = siloDepositEventsDbTimedClean['dateTime'].dt.normalize()
siloDepositEventsDbTimedClean.head(100)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
6,deposit,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,16646360,1676611691,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,10.000000,10.66869,105.925340,2023-02-17 05:28:11,2023-02-17
36,deposit,0xf9bbcc923182fb6406e97fce0f92c22c87a284d55812...,16627144,1676379599,0x245cc372c84b3645bf0ffe6538620b04a217988b,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,20000.000000,10.66869,206726.334541,2023-02-14 12:59:59,2023-02-14
42,deposit,0x0aa351eb8ff1ca2bd6c4e65229f83c7e5b596d72c889...,16625541,1676360171,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,2.514698,10.66869,26.317329,2023-02-14 07:36:11,2023-02-14


In [423]:
path =file+'/siloFinanceGohmDeposits.csv'
siloDepositEventsDbTimedClean.to_csv(path, index = False)

path =file+'/siloFinanceGohmDeposits_piped.csv'
siloDepositEventsDbTimedClean.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(siloDepositEventsDbTimedClean),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\ohmBorrowing/data/siloFinanceGohmDeposits_piped.csv file saved, 3  records


In [424]:
totalDeposited = sum(siloDepositEventsDbTimedClean['transactionReadableAmount'])
print('There has been a total of',round(totalDeposited,2),  'OHM deposited cumulatively as collateral  to date')

There has been a total of 20012.51 OHM deposited cumulatively as collateral  to date


In [425]:
totalDepositedUSD = sum(siloDepositEventsDbTimedClean['transaction_amountUSD'])
print('That is a value of USD$',round(totalDepositedUSD,2))

That is a value of USD$ 206858.58


********************************************************************
********************************************************************
Now we need withdrawals to counter the deposits and see current state

In [426]:
oldSiloWithdrawsDb = pd.DataFrame()
siloFileName = file+'/siloFinanceGohmWithdraws.csv'
try:
    oldSiloWithdrawsDb = pd.read_csv (siloFileName)
except:
    oldSiloWithdrawsDb = pd.DataFrame()
oldSiloWithdrawsDb.head(100)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,withdraw,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,16627397,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,2.514698,10.66869,25.992718,2023-02-14 13:50:59,2023-02-14


In [427]:
maxSiloWithdrawsTimestamp = 0
try:
    maxSiloWithdrawsTimestamp = (max(0,oldSiloWithdrawssDb['transaction_timestamp'].max()))-5
except:
        maxSiloWithdrawsTimestamp = 0
maxSiloWithdrawsTimestamp = int(maxSiloWithdrawsTimestamp)
maxSiloWithdrawsTimestamp

0

In [428]:
##this captures the ENTIRE list of people who withdrawed OHM
withdrawTicker = 0
withdrawslist = pd.DataFrame()
siloWithdrawsDb = pd.DataFrame()
withdrawListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (withdrawTicker)*(1000)
    withdraws = silo.Query.withdraws(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db'
                ,'timestamp_gte': maxSiloWithdrawsTimestamp
                ,'asset':'0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5'
                ,'amount_gte': 0
                } #gOHM market
    )

    withdrawslist = sg.query_df([
        withdraws.id
        ,withdraws.hash
        ,withdraws.blockNumber
        ,withdraws.timestamp
        ,withdraws.account.id
        ,withdraws.market.id
        ,withdraws.market.protocol.id
        ,withdraws.market.rates.rate
        ,withdraws.market.rates.duration
        ,withdraws.market.rates.side
        ,withdraws.market.rates.type
        ,withdraws.market.rates.token.id
        ,withdraws.asset.name
        ,withdraws.asset.symbol
        ,withdraws.asset.decimals
        ,withdraws.asset.lastPriceUSD
        ,withdraws.amount
        ,withdraws.amountUSD
    ])

    siloWithdrawsDb=pd.concat([siloWithdrawsDb, withdrawslist])
    recordTimestamp1 = siloWithdrawsDb.iat[withdrawTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1)
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    silowithdrawsDbLength = len(siloWithdrawsDb)
    silowithdrawListLength = len(withdrawslist)
    recordID = siloWithdrawsDb.iat[withdrawTicker,0]
    withdraw = siloWithdrawsDb.iat[withdrawTicker,1]
    print("iterations: ", withdrawTicker, "Lines skipped: ",skipValue, "records collected: ", silowithdrawListLength, " - latest recordID: ",withdraw, " ", recordID , " - withdraws DB length: ", silowithdrawsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", withdrawTicker, "records collected: ", withdrawListLength, " - userdb length: ", withdrawsDbLength)
    withdrawTicker = withdrawTicker+1

    if daysAgo>daysLimit: exit =True
    if silowithdrawListLength<1000: exit =True

siloWithdrawsDb.columns = siloWithdrawsDb.columns.str.replace("withdraws", "transaction")
siloWithdrawsDb['event']='withdraw'
siloWithdrawsDb.drop_duplicates
print("Done. Total Records collected: ", silowithdrawsDbLength)

iterations:  0 Lines skipped:  0 records collected:  6  - latest recordID:  0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7f23b9e1bb32b1fac6ec0   0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7f23b9e1bb32b1fac6ec0-259  - withdraws DB length:  6  - latest record from:  2023-02-14 05:50:59 5 days ago
Done. Total Records collected:  6


In [429]:
siloWithdrawEventsDbTimedClean = pd.DataFrame()
siloWithdrawsDb['transactionReadableAmount'] = siloWithdrawsDb['transaction_amount']/(10**(siloWithdrawsDb['transaction_asset_decimals']))
siloWithdrawsDb['transactionReadableAmount']=siloWithdrawsDb['transactionReadableAmount'].astype('float')
siloWithdrawsDb['dateTime'] =(pd.to_datetime(siloWithdrawsDb['transaction_timestamp'],unit='s'))

#we only want withdrawals in GOHM from those that borrowed, so drop anyone that is not on that side
siloWithdrawsDb.drop(siloWithdrawsDb.loc[siloWithdrawsDb['transaction_market_rates_side']!='BORROWER'].index, inplace=True)
siloWithdrawsDb.drop(siloWithdrawsDb.loc[siloWithdrawsDb['transaction_asset_symbol']!='OHM'].index, inplace=True)

siloWithdrawEventsDbTimedClean = siloWithdrawsDb[['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime']].copy()
siloWithdrawEventsDbTimedClean.sort_values(by=['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime'])
siloWithdrawEventsDbTimedClean.drop_duplicates(subset=['transaction_hash'],keep='first', inplace=True)
siloWithdrawEventsDbTimedClean['calDate'] = siloWithdrawEventsDbTimedClean['dateTime'].dt.normalize()
siloWithdrawsDb.head(100)

,transaction_id,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_market_protocol_id,transaction_market_rates_rate,transaction_market_rates_duration,transaction_market_rates_side,...,transaction_market_rates_token_id,transaction_asset_name,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,transaction_amountUSD,event,transactionReadableAmount,dateTime
0,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,16627397,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,1.164533,None,BORROWER,...,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,Olympus,OHM,9,10.66869,2514698283,25.992718,withdraw,2.514698,2023-02-14 13:50:59
1,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,16627397,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Olympus,OHM,9,10.66869,2514698283,25.992718,withdraw,2.514698,2023-02-14 13:50:59
2,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,16627397,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,...,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Olympus,OHM,9,10.66869,2514698283,25.992718,withdraw,2.514698,2023-02-14 13:50:59


In [430]:
path =file+'/siloFinanceGohmWithdraws.csv'
siloWithdrawEventsDbTimedClean.to_csv(path, index = False)

path =file+'/siloFinanceGohmWithdraws_piped.csv'
siloWithdrawEventsDbTimedClean.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(siloWithdrawEventsDbTimedClean),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\ohmBorrowing/data/siloFinanceGohmWithdraws_piped.csv file saved, 1  records


In [431]:
totalWithdrawed = sum(siloWithdrawEventsDbTimedClean['transactionReadableAmount'])
print('There has been a total of',round(totalWithdrawed,2),  'OHM withdrawn cumulatively as collateral  to date')
totalWithdrawedUSD =sum(siloWithdrawEventsDbTimedClean['transaction_amountUSD'])
print('That is a value of USD$',round(totalWithdrawedUSD,2))

There has been a total of 2.51 OHM withdrawn cumulatively as collateral  to date
That is a value of USD$ 25.99


In [433]:
collateralBalance = totalDeposited-totalWithdrawed
print(totalDeposited, ' - ',totalWithdrawed,'=', collateralBalance, 'currently deposited as collateral in Silo' )

20012.514698283  -  2.514698283 = 20010.0 currently deposited as collateral in Silo


*************************************************************
*************************************************************
LET'S LOOK AT THE EULER MARKET
https://app.euler.finance/market/0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5
https://gateway.thegraph.com/api/[api-key]/subgraphs/id/8cLf29KxAedWLVaEqjV8qKomdwwXQxjptBZFrqWNH5u2

STARTING WITH BORROWS (hasn't launched yet as of 2/16/23)

In [434]:
euler = sg.load_api('https://gateway.thegraph.com/api/'+apiKey+'/subgraphs/id/8cLf29KxAedWLVaEqjV8qKomdwwXQxjptBZFrqWNH5u2')

In [435]:
oldEulerBorrowsDb = pd.DataFrame()
EulerBorrowsFileName = file+'/eulerFinanceOhmBorrows.csv'
try:
    oldEulerBorrowsDb = pd.read_csv (EulerBorrowsFileName)
except:
    oldEulerBorrowsDb = pd.DataFrame()
oldEulerBorrowsDb.head(100)

""


In [436]:
maxEulerBorrowTimestamp = 0
try:
    maxEulerBorrowTimestamp = (max(0,oldBorrowsDb['transaction_timestamp'].max()))-5
except:
        maxEulerBorrowTimestamp = 0
maxEulerBorrowTimestamp = int(maxEulerBorrowTimestamp)
maxEulerBorrowTimestamp

0

##SAMPLE QUERY
eulerBorrowsDb = pd.DataFrame()
deposits = euler.Query.deposits(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ #'timestamp_gte': maxEulerdepositTimestamp
                #,'hash': '0x301069fd24dd70d15cdcbebd4f076efe486a82f347240703adcb1713a6582e98'
                'market':'0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b'
                ,'asset':'0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5'
                }
    )

depositslist = sg.query_df([
    deposits.id
    ,deposits.hash
    ,deposits.logIndex
    ,deposits.timestamp
    ,deposits.to
    ,deposits.asset.id
    ,deposits.asset.symbol
    ,deposits.asset.decimals
    ,deposits.asset.lastPriceUSD
    ,deposits.asset.lastPriceBlockNumber
    ,deposits.amount
    ,deposits.amountUSD
    ,deposits._select("from")
])

depositslist.head(10)

In [437]:
##this captures the ENTIRE list of people who borrowed OHM
borrowTicker = 0
borrowslist = pd.DataFrame()
eulerBorrowsDb = pd.DataFrame()
eulerBorrowsDbLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (borrowTicker)*(1000)
    borrows = euler.Query.borrows(
            orderBy='timestamp',
            orderDirection='desc',
            first=1000,
            skip = skipValue,
            where={ #'timestamp_gte': maxEulerBorrowTimestamp
                    #,'hash': '0x301069fd24dd70d15cdcbebd4f076efe486a82f347240703adcb1713a6582e98'
                    'market':'0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b'
                    ,'asset':'0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5'
                    }
        )

    borrowslist = sg.query_df([
        borrows.id
        ,borrows.hash
        ,borrows.logIndex
        ,borrows.timestamp
        ,borrows.to
        ,borrows.asset.id
        ,borrows.asset.symbol
        ,borrows.asset.decimals
        ,borrows.asset.lastPriceUSD
        ,borrows.asset.lastPriceBlockNumber
        ,borrows.amount
        ,borrows.amountUSD
        ,borrows._select("from")
    ])

    eulerBorrowsDb=pd.concat([eulerBorrowsDb, borrowslist]).

    recordTimestamp1 = eulerBorrowsDb.iat[borrowTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    eulerBorrowsDbLength = len(eulerBorrowsDb)
    eulerBorrowListLength = len(borrowslist)
    recordID = eulerBorrowsDb.iat[borrowTicker,0]
    borrow = eulerBorrowsDb.iat[borrowTicker,1]
    print("iterations: ", borrowTicker, "Lines skipped: ",skipValue, "records collected: ", eulerBorrowListLength, " - latest recordID: ",borrow, " ", recordID , " - borrows DB length: ", eulerBorrowsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", borrowTicker, "records collected: ", borrowListLength, " - userdb length: ", borrowsDbLength)
    borrowTicker = borrowTicker+1

    if daysAgo>daysLimit: exit =True
    if eulerBorrowListLength<1000: exit =True

eulerBorrowsDb.columns = eulerBorrowsDb.columns.str.replace("borrows", "transaction")
eulerBorrowsDb['event']='borrow'
eulerBorrowsDb.drop_duplicates
print("Done. Total Records collected: ", eulerBorrowsDbLength)

SyntaxError: invalid syntax (1674637684.py, line 41)

In [ ]:
eulerBorrowsDb.head(10)

****************************************************************************************************************
****************************************************************************************************************
LET'S LOOK AT THE EULER DEPOSIT EVENTS

In [505]:
oldEulerDepositsDb = pd.DataFrame()
EulerDepositsFileName = file+'/eulerFinanceOhmDeposits.csv'
try:
    oldEulerDepositsDb = pd.read_csv (EulerDepositsFileName)
except:
    oldEulerDepositsDb = pd.DataFrame()
oldEulerDepositsDb.head(100)

,deposit_id,deposit_hash,deposit_logIndex,deposit_timestamp,deposit_to,deposit_asset_id,deposit_asset_symbol,deposit_asset_decimals,deposit_asset_lastPriceUSD,deposit_asset_lastPriceBlockNumber,deposit_amount,deposit_amountUSD,deposit_from,event,depositReadableAmount,dateTime
0,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,199,1676615723,0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,OHM,9,10.66869,16665524,4973203839,52.678831,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,deposit,4.973204,2023-02-17 06:35:23
1,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,123,1676379695,0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,OHM,9,10.66869,16665524,30000000000000,310089.501811,0x245cc372c84b3645bf0ffe6538620b04a217988b,deposit,30000.000000,2023-02-14 13:01:35


In [508]:
maxEulerDepositTimestamp = 0
try:
    maxEulerDepositTimestamp = (max(0,oldEulerDepositsDb['deposit_timestamp'].max()))-5
except:
        maxEulerDepositTimestamp = 0
maxEulerDepositTimestamp = int(maxEulerDepositTimestamp)
maxEulerDepositTimestamp

1676615718

In [509]:
##this captures the ENTIRE list of people who Deposited OHM
depositTicker = 0
depositlist = pd.DataFrame()
eulerDepositsDb = pd.DataFrame()
eulerDepositsDbLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (depositTicker)*(1000)
    deposits = euler.Query.deposits(
            orderBy='timestamp',
            orderDirection='desc',
            first=1000,
            skip = skipValue,
            where={ #'timestamp_gte': maxEulerDepositTimestamp
                    #,'hash': '0x301069fd24dd70d15cdcbebd4f076efe486a82f347240703adcb1713a6582e98'
                    'market':'0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b'
                    ,'asset':'0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5'
                    }
        )

    depositslist = sg.query_df([
        deposits.id
        ,deposits.hash
        ,deposits.logIndex
        ,deposits.timestamp
        ,deposits.to
        ,deposits.asset.id
        ,deposits.asset.symbol
        ,deposits.asset.decimals
        ,deposits.asset.lastPriceUSD
        ,deposits.asset.lastPriceBlockNumber
        ,deposits.amount
        ,deposits.amountUSD
        ,deposits._select("from")
    ])

    eulerDepositsDb=pd.concat([eulerDepositsDb, depositslist])

    recordTimestamp1 = eulerDepositsDb.iat[depositTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    eulerDepositsDbLength = len(eulerDepositsDb)
    eulerDepositListLength = len(depositslist)
    recordID = eulerDepositsDb.iat[depositTicker,0]
    deposit = eulerDepositsDb.iat[depositTicker,1]
    print("iterations: ", depositTicker, "Lines skipped: ",skipValue, "records collected: ", eulerDepositListLength, " - latest recordID: ",deposit, " ", recordID , " - deposits DB length: ", eulerDepositsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", depositTicker, "records collected: ", depositListLength, " - userdb length: ", depositsDbLength)
    depositTicker = depositTicker+1

    if daysAgo>daysLimit: exit =True
    if eulerDepositListLength<1000: exit =True

eulerDepositsDb.columns = eulerDepositsDb.columns.str.replace("deposits", "transaction")
eulerDepositsDb['event']='deposit'
eulerDepositsDb.drop_duplicates
print("Done. Total Records collected: ", eulerDepositsDbLength)

iterations:  0 Lines skipped:  0 records collected:  2  - latest recordID:  0x301069fd24dd70d15cdcbebd4f076efe486a82f347240703adcb1713a6582e98   0x301069fd24dd70d15cdcbebd4f076efe486a82f347240703adcb1713a6582e98-199  - deposits DB length:  2  - latest record from:  2023-02-16 22:35:23 3 days ago
Done. Total Records collected:  2


In [510]:
eulerDepositsDb.head(10)

,transaction_id,transaction_hash,transaction_logIndex,transaction_timestamp,transaction_to,transaction_asset_id,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_asset_lastPriceBlockNumber,transaction_amount,transaction_amountUSD,transaction_from,event
0,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,199,1676615723,0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,OHM,9,10.667133,16667356,4973203839,52.678831,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,deposit
1,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,123,1676379695,0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,OHM,9,10.667133,16667356,30000000000000,310089.501811,0x245cc372c84b3645bf0ffe6538620b04a217988b,deposit


In [511]:
eulerdepositsDbTimedClean = pd.DataFrame()
eulerDepositsDb['transactionReadableAmount'] = eulerDepositsDb['transaction_amount']/(10**(eulerDepositsDb['transaction_asset_decimals']))

eulerDepositsDb['transactionReadableAmount']=eulerDepositsDb['transactionReadableAmount'].astype('float')
eulerDepositsDb['dateTime'] =(pd.to_datetime(eulerDepositsDb['transaction_timestamp'],unit='s'))
eulerDepositsDb.head(10)

,transaction_id,transaction_hash,transaction_logIndex,transaction_timestamp,transaction_to,transaction_asset_id,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_asset_lastPriceBlockNumber,transaction_amount,transaction_amountUSD,transaction_from,event,transactionReadableAmount,dateTime
0,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,199,1676615723,0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,OHM,9,10.667133,16667356,4973203839,52.678831,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,deposit,4.973204,2023-02-17 06:35:23
1,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,123,1676379695,0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,OHM,9,10.667133,16667356,30000000000000,310089.501811,0x245cc372c84b3645bf0ffe6538620b04a217988b,deposit,30000.000000,2023-02-14 13:01:35


In [512]:
eulerDepositsDb['transactionReadableAmount'] = eulerDepositsDb['transaction_amount']*1e-9
eulerDepositsDb['dateTime'] =(pd.to_datetime(eulerDepositsDb['transaction_timestamp'],unit='s'))
eulerDepositsDb.head(10)

,transaction_id,transaction_hash,transaction_logIndex,transaction_timestamp,transaction_to,transaction_asset_id,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_asset_lastPriceBlockNumber,transaction_amount,transaction_amountUSD,transaction_from,event,transactionReadableAmount,dateTime
0,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,199,1676615723,0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,OHM,9,10.667133,16667356,4973203839,52.678831,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,deposit,4.973204,2023-02-17 06:35:23
1,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,123,1676379695,0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,OHM,9,10.667133,16667356,30000000000000,310089.501811,0x245cc372c84b3645bf0ffe6538620b04a217988b,deposit,30000.000000,2023-02-14 13:01:35


In [439]:
eulerDepositsDb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 16 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   deposit_id                          2 non-null      object        
 1   deposit_hash                        2 non-null      object        
 2   deposit_logIndex                    2 non-null      int64         
 3   deposit_timestamp                   2 non-null      int64         
 4   deposit_to                          2 non-null      object        
 5   deposit_asset_id                    2 non-null      object        
 6   deposit_asset_symbol                2 non-null      object        
 7   deposit_asset_decimals              2 non-null      int64         
 8   deposit_asset_lastPriceUSD          2 non-null      float64       
 9   deposit_asset_lastPriceBlockNumber  2 non-null      int64         
 10  deposit_amount                

In [513]:
totalDepositedEuler = sum(eulerDepositsDb['transactionReadableAmount'])
print('There has been a total of', round(totalDepositedEuler, 2), 'OHM deposited cumulatively to date')

There has been a total of 30004.97 OHM deposited cumulatively to date


In [514]:
totalDepositedUSDEuler = sum(eulerDepositsDb['transaction_amountUSD'])
print('That\'s $', round(totalDepositedUSDEuler, 2), 'USD deposited at last price')

That's $ 310142.18 USD deposited at last price


In [515]:
path =file+'/eulerFinanceOhmDeposits.csv'
eulerDepositsDb.to_csv(path, index = False)

path =file+'/eulerFinanceOhmDeposits_piped.csv'
depositsDb.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(siloWithdrawEventsDbTimedClean),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\ohmBorrowing/data/eulerFinanceOhmDeposits_piped.csv file saved, 1  records


****************************************************************************************************************
****************************************************************************************************************
EULER WITHDRAWAL EVENTS (none as of 02/19)

In [516]:
oldEulerWithdrawsDb = pd.DataFrame()
EulerFileName = file+'/EulerFinanceGohmWithdraws.csv'
try:
    oldEulerWithdrawsDb = pd.read_csv (EulerFileName)
except:
    oldEulerWithdrawsDb = pd.DataFrame()
oldEulerWithdrawsDb.head(100)

""


In [517]:
maxEulerWithdrawsTimestamp = 0
try:
    maxEulerWithdrawsTimestamp = (max(0,oldEulerWithdrawssDb['transaction_timestamp'].max()))-5
except:
        maxEulerWithdrawsTimestamp = 0
maxEulerWithdrawsTimestamp = int(maxEulerWithdrawsTimestamp)
maxEulerWithdrawsTimestamp

0

In [518]:
##this captures the ENTIRE list of people who withdrawed OHM
try:
    withdrawTicker = 0
    withdrawslist = pd.DataFrame()
    eulerWithdrawsDb = pd.DataFrame()
    withdrawListLength = 1000
    datediff = 0
    now=0
    daysAgo=0
    daysLimit = 45
    exit = False
    while exit==False:
        skipValue = (withdrawTicker)*(1000)
        withdraws = euler.Query.withdraws(
            orderBy='timestamp',
            orderDirection='desc',
            first=1000,
            skip = skipValue,
            where={ #'timestamp_gte': maxEulerBorrowTimestamp
                    #,'hash': '0x301069fd24dd70d15cdcbebd4f076efe486a82f347240703adcb1713a6582e98'
                    'market':'0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b'
                    ,'asset':'0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5'
                    }
            )

        withdrawslist = sg.query_df([
            withdraws.id
            ,withdraws.hash
            ,withdraws.logIndex
            ,withdraws.timestamp
            ,withdraws.to
            ,withdraws.asset.id
            ,withdraws.asset.symbol
            ,withdraws.asset.decimals
            ,withdraws.asset.lastPriceUSD
            ,withdraws.asset.lastPriceBlockNumber
            ,withdraws.amount
            ,withdraws.amountUSD
            ,withdraws._select("from")
        ])

        eulerWithdrawsDb=pd.concat([eulerWithdrawsDb, withdrawslist])
        recordTimestamp1 = eulerWithdrawsDb.iat[withdrawTicker,3]
        recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
        now = (int(dt.datetime.utcnow().timestamp()))
        datediff=abs(int(now) - recordTimestamp1)
        daysAgo = int(datediff/86400)
        eulerwithdrawsDbLength = len(eulerWithdrawsDb)
        eulerwithdrawListLength = len(withdrawslist)
        recordID = eulerWithdrawsDb.iat[withdrawTicker,0]
        withdraw = eulerWithdrawsDb.iat[withdrawTicker,1]
        print("iterations: ", withdrawTicker, "Lines skipped: ",skipValue, "records collected: ", eulerwithdrawListLength, " - latest recordID: ",withdraw, " ", recordID , " - withdraws DB length: ", eulerwithdrawsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
        #print("iterations: ", withdrawTicker, "records collected: ", withdrawListLength, " - userdb length: ", withdrawsDbLength)
        withdrawTicker = withdrawTicker+1

        if daysAgo>daysLimit: exit =True
        if withdrawListLength<1000: exit =True

    eulerWithdrawsDb.columns = eulerWithdrawsDb.columns.str.replace("withdraws", "transaction")
    eulerWithdrawsDb['event']='withdraw'
    eulerWithdrawsDb.drop_duplicates
    print("Done. Total Records collected: ", eulerwithdrawsDbLength)
except:
    eulerWithdrawsDb =pd.DataFrame()
    eulerWithdrawsDb['withdraws_id']=[]
    eulerWithdrawsDb['withdraws_hash']=[]
    eulerWithdrawsDb['withdraws_logIndex']=[]
    eulerWithdrawsDb['withdraws_timestamp']=[]
    eulerWithdrawsDb['withdraws_to']=[]
    eulerWithdrawsDb['withdraws_asset_id']=[]
    eulerWithdrawsDb['withdraws_asset_symbol']=[]
    eulerWithdrawsDb['withdraws_asset_decimals']=[]
    eulerWithdrawsDb['withdraws_asset_lastPriceUSD']=[]
    eulerWithdrawsDb['withdraws_asset_lastPriceBlockNumber']=[]
    eulerWithdrawsDb['withdraws_amount']=[]
    eulerWithdrawsDb['withdraws_amountUSD']=[]
    eulerWithdrawsDb['withdraws_from']=[]
eulerWithdrawsDb.head(10)

,withdraws_id,withdraws_hash,withdraws_logIndex,withdraws_timestamp,withdraws_to,withdraws_asset_id,withdraws_asset_symbol,withdraws_asset_decimals,withdraws_asset_lastPriceUSD,withdraws_asset_lastPriceBlockNumber,withdraws_amount,withdraws_amountUSD,withdraws_from


In [519]:
eulerWithdrawEventsDbTimedClean = pd.DataFrame()
eulerWithdrawsDb['transactionReadableAmount'] = eulerWithdrawsDb['withdraws_amount']/(10**(eulerWithdrawsDb['withdraws_asset_decimals']))

eulerWithdrawsDb['transactionReadableAmount']=eulerWithdrawsDb['transactionReadableAmount'].astype('float')
eulerWithdrawsDb['dateTime'] =(pd.to_datetime(eulerWithdrawsDb['withdraws_timestamp'],unit='s'))
 

eulerWithdrawEventsDbTimedClean = eulerWithdrawsDb['withdraws_id','withdraws_hash','withdraws_logIndex','withdraws_to','withdraws_asset_id','withdraws_asset_symbol','withdraws_asset_decimals', 'withdraws_asset_lastPriceUSD','withdraws_asset_lastPriceBlockNumber','withdraws_amount','withdraws_from','dateTime'].copy()
eulerWithdrawEventsDbTimedClean.sort_values(by=['withdraws_id','withdraws_hash','withdraws_logIndex','withdraws_to','withdraws_asset_id','withdraws_asset_symbol','withdraws_asset_decimals', 'withdraws_asset_lastPriceUSD','withdraws_asset_lastPriceBlockNumber','withdraws_amount','withdraws_from','dateTime'])
eulerWithdrawEventsDbTimedClean.drop_duplicates(subset=['withdraws_hash'],keep='first', inplace=True)
eulerWithdrawEventsDbTimedClean['calDate'] = eulerWithdrawEventsDbTimedClean['dateTime'].dt.normalize()
eulerWithdrawsDb.head(100)

KeyError: ('withdraws_id', 'withdraws_hash', 'withdraws_logIndex', 'withdraws_to', 'withdraws_asset_id', 'withdraws_asset_symbol', 'withdraws_asset_decimals', 'withdraws_asset_lastPriceUSD', 'withdraws_asset_lastPriceBlockNumber', 'withdraws_amount', 'withdraws_from', 'dateTime')

****************************************************************************************************************
****************************************************************************************************************
COMBINE DEPOSIT EVENTS

In [530]:
#COLLATERALDEPOSITS
eulerDepositsDbPreMerge = db.query(
    "select distinct"
    " transaction_hash actions_id "
    ", transaction_timestamp actions_timestamp "
    ", dateTime "
    ", transaction_from transaction_from "
    ", transaction_asset_symbol asset_symbol"
    ", transactionReadableAmount ReadableAmount "
    ", transaction_amountUSD transaction_amountUSD "
    "from eulerDepositsDb "
).df()
eulerDepositsDbPreMerge['protocol'] = 'Euler'
eulerDepositsDbPreMerge['action'] = 'depositCollateral'
eulerDepositsDbPreMerge.head(10)



,actions_id,actions_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,transaction_amountUSD,protocol,action
0,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,1676615723,2023-02-17 06:35:23,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,4.973204,52.678831,Euler,depositCollateral
1,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,1676379695,2023-02-14 13:01:35,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,30000.000000,310089.501811,Euler,depositCollateral


In [531]:
siloDepositEventsDbTimedClean.head(1)

,event,deposits_hash,deposits_blockNumber,deposits_timestamp,deposits_account_id,deposits_market_id,deposits_asset_name,deposits_asset_symbol,depositsReadableAmount,deposits_asset_lastPriceUSD,deposits_amountUSD,dateTime,calDate
6,deposit,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,16646360,1676611691,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,10.0,10.66869,105.92534,2023-02-17 05:28:11,2023-02-17


In [602]:
siloDepositsDbPreMerge = db.query(
    "select distinct"
    " deposits_hash actions_id "
    ", deposits_timestamp actions_timestamp "
    ", dateTime "
    ", deposits_account_id transaction_from "
    ", deposits_asset_symbol asset_symbol "
    ", depositsReadableAmount ReadableAmount "
    ", deposits_amountUSD transaction_amountUSD "
    "from siloDepositEventsDbTimedClean "
).df()
siloDepositsDbPreMerge['protocol'] = 'Silo'
siloDepositsDbPreMerge['action'] = 'depositCollateral'
siloDepositsDbPreMerge.head(10)


,actions_id,actions_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,transaction_amountUSD,protocol,action
0,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,1676611691,2023-02-17 05:28:11,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,10.000000,105.925340,Silo,depositCollateral
1,0xf9bbcc923182fb6406e97fce0f92c22c87a284d55812...,1676379599,2023-02-14 12:59:59,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,20000.000000,206726.334541,Silo,depositCollateral
2,0x0aa351eb8ff1ca2bd6c4e65229f83c7e5b596d72c889...,1676360171,2023-02-14 07:36:11,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,26.317329,Silo,depositCollateral


In [603]:
ohmDeposits = pd.concat([eulerDepositsDbPreMerge,siloDepositsDbPreMerge])
ohmDeposits.head(100)

,actions_id,actions_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,transaction_amountUSD,protocol,action
0,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,1676615723,2023-02-17 06:35:23,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,4.973204,52.678831,Euler,depositCollateral
1,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,1676379695,2023-02-14 13:01:35,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,30000.000000,310089.501811,Euler,depositCollateral
0,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,1676611691,2023-02-17 05:28:11,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,10.000000,105.925340,Silo,depositCollateral
1,0xf9bbcc923182fb6406e97fce0f92c22c87a284d55812...,1676379599,2023-02-14 12:59:59,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,20000.000000,206726.334541,Silo,depositCollateral
2,0x0aa351eb8ff1ca2bd6c4e65229f83c7e5b596d72c889...,1676360171,2023-02-14 07:36:11,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,26.317329,Silo,depositCollateral


In [604]:
path =file+'/allOhmDeposits.csv'
ohmDeposits.to_csv(path, index = False)

path =file+'/allOhmDeposits_piped.csv'
ohmDeposits.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(ohmDeposits),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\ohmBorrowing/data/allOhmDeposits_piped.csv file saved, 5  records


****************************************************************************************************************
****************************************************************************************************************
COMBINE WITHDRAWAL EVENTS

In [605]:
siloWithdrawsDb['protocol'] = 'Silo'
siloWithdrawsDb.head(1)

,transaction_id,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_market_protocol_id,transaction_market_rates_rate,transaction_market_rates_duration,transaction_market_rates_side,...,transaction_asset_name,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,transaction_amountUSD,event,transactionReadableAmount,dateTime,protocol
0,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,16627397,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,1.164533,None,BORROWER,...,Olympus,OHM,9,10.66869,2514698283,25.992718,withdraw,2.514698,2023-02-14 13:50:59,Silo


In [606]:
allCollateralWithdrawals = pd.concat([siloWithdrawsDb])
allCollateralWithdrawals.head(10)

,transaction_id,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_market_protocol_id,transaction_market_rates_rate,transaction_market_rates_duration,transaction_market_rates_side,...,transaction_asset_name,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,transaction_amountUSD,event,transactionReadableAmount,dateTime,protocol
0,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,16627397,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,1.164533,None,BORROWER,...,Olympus,OHM,9,10.66869,2514698283,25.992718,withdraw,2.514698,2023-02-14 13:50:59,Silo
1,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,16627397,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,...,Olympus,OHM,9,10.66869,2514698283,25.992718,withdraw,2.514698,2023-02-14 13:50:59,Silo
2,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,16627397,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,...,Olympus,OHM,9,10.66869,2514698283,25.992718,withdraw,2.514698,2023-02-14 13:50:59,Silo


****************************************************************************************************************
****************************************************************************************************************
COMBINE WITHDRAWAL AND DEPOSIT EVENTS

In [607]:
ohmDeposited.columns = ohmDeposited.columns.str.replace("deposit", "transaction")
ohmDeposited.head(1)

,transaction_hash,transaction_timestamp,transaction_from,transaction_asset_symbol,transactionReadableAmount,transaction_amountUSD,action,protocol
0,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,1676615723,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,4.973204,52.678831,depositCollateral,Euler


In [608]:
allCollateralWithdrawals = db.query("select "
                                    "transaction_hash "
                                    ",transaction_timestamp "
                                    ", transaction_account_id  transaction_from "
                                    ", transaction_asset_symbol "
                                    ", transactionReadableAmount "
                                    ", transaction_amountUSD "
                                    ", 'withdrawCollateral' as action "
                                    ", protocol "
                                    "from allCollateralWithdrawals "
                                    "").df()
allCollateralWithdrawals.head(1)

,transaction_hash,transaction_timestamp,transaction_from,transaction_asset_symbol,transactionReadableAmount,transaction_amountUSD,action,protocol
0,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,25.992718,withdrawCollateral,Silo


In [609]:
allOhmBorrowMarketEvents = pd.concat([ohmDeposited,allCollateralWithdrawals])
allOhmBorrowMarketEvents['dateTime'] =(pd.to_datetime(allOhmBorrowMarketEvents['transaction_timestamp'],unit='s'))
allOhmBorrowMarketEvents.head(100)

,transaction_hash,transaction_timestamp,transaction_from,transaction_asset_symbol,transactionReadableAmount,transaction_amountUSD,action,protocol,dateTime
0,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,1676615723,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,4.973204,52.678831,depositCollateral,Euler,2023-02-17 06:35:23
1,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,1676379695,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,30000.000000,310089.501811,depositCollateral,Euler,2023-02-14 13:01:35
0,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,1676611691,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,10.000000,105.925340,depositCollateral,Silo,2023-02-17 05:28:11
1,0xf9bbcc923182fb6406e97fce0f92c22c87a284d55812...,1676379599,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,20000.000000,206726.334541,depositCollateral,Silo,2023-02-14 12:59:59
2,0x0aa351eb8ff1ca2bd6c4e65229f83c7e5b596d72c889...,1676360171,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,26.317329,depositCollateral,Silo,2023-02-14 07:36:11
0,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,25.992718,withdrawCollateral,Silo,2023-02-14 13:50:59
1,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,25.992718,withdrawCollateral,Silo,2023-02-14 13:50:59
2,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,25.992718,withdrawCollateral,Silo,2023-02-14 13:50:59


****************************************************************************************************************
****************************************************************************************************************
COMBINE BORROW EVENTS

In [610]:
siloBorrowEventsDbTimedClean.head(200)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,borrow,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,16648400,1676636519,0xcb6e1613029d790c00f89296808f278d6dc25b2f,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,58.000000,10.66869,615.628222,2023-02-17 12:21:59,2023-02-17
6,borrow,0x6726ea9fa581ab0a5fa26b8a112c95e690500f446c39...,16628130,1676391467,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,1480.000000,10.66869,15782.938441,2023-02-14 16:17:47,2023-02-14
12,borrow,0x5fca09aaf64619a5d644ed392a64f5bd08b9e6b4b46e...,16627404,1676382743,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,175.320428,10.66869,1812.167469,2023-02-14 13:52:23,2023-02-14


In [619]:
siloBorrowsDbPreMerge = db.query(
    "select distinct"
    " transaction_hash  "
    ", transaction_timestamp transaction_timestamp "
    ", transaction_account_id transaction_from "
    ", transaction_asset_symbol  "
    ", transactionReadableAmount transactionReadableAmount "
    ", transaction_amountUSD transaction_amountUSD "
    "from siloBorrowEventsDbTimedClean "
).df()
siloBorrowsDbPreMerge['action'] = 'borrowAsset'
siloBorrowsDbPreMerge['protocol'] = 'Silo'
siloBorrowsDbPreMerge.head(10)


,transaction_hash,transaction_timestamp,transaction_from,transaction_asset_symbol,transactionReadableAmount,transaction_amountUSD,action,protocol
0,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,1676636519,0xcb6e1613029d790c00f89296808f278d6dc25b2f,OHM,58.000000,615.628222,borrowAsset,Silo
1,0x6726ea9fa581ab0a5fa26b8a112c95e690500f446c39...,1676391467,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,OHM,1480.000000,15782.938441,borrowAsset,Silo
2,0x5fca09aaf64619a5d644ed392a64f5bd08b9e6b4b46e...,1676382743,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,175.320428,1812.167469,borrowAsset,Silo


EVENTUALLY THERE WILL BE EULER BORROWS

In [620]:
allOhmBorrowEvents = pd.concat([siloBorrowsDbPreMerge])
allOhmBorrowEvents.head(10)

,transaction_hash,transaction_timestamp,transaction_from,transaction_asset_symbol,transactionReadableAmount,transaction_amountUSD,action,protocol
0,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,1676636519,0xcb6e1613029d790c00f89296808f278d6dc25b2f,OHM,58.000000,615.628222,borrowAsset,Silo
1,0x6726ea9fa581ab0a5fa26b8a112c95e690500f446c39...,1676391467,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,OHM,1480.000000,15782.938441,borrowAsset,Silo
2,0x5fca09aaf64619a5d644ed392a64f5bd08b9e6b4b46e...,1676382743,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,175.320428,1812.167469,borrowAsset,Silo


In [621]:
allCollateralWithdrawals.head(10)

,transaction_hash,transaction_timestamp,transaction_from,transaction_asset_symbol,transactionReadableAmount,transaction_amountUSD,action,protocol
0,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,25.992718,withdrawCollateral,Silo
1,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,25.992718,withdrawCollateral,Silo
2,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,25.992718,withdrawCollateral,Silo


In [622]:
ohmDeposited.head(10)

,transaction_hash,transaction_timestamp,transaction_from,transaction_asset_symbol,transactionReadableAmount,transaction_amountUSD,action,protocol
0,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,1676615723,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,4.973204,52.678831,depositCollateral,Euler
1,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,1676379695,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,30000.000000,310089.501811,depositCollateral,Euler
0,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,1676611691,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,10.000000,105.925340,depositCollateral,Silo
1,0xf9bbcc923182fb6406e97fce0f92c22c87a284d55812...,1676379599,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,20000.000000,206726.334541,depositCollateral,Silo
2,0x0aa351eb8ff1ca2bd6c4e65229f83c7e5b596d72c889...,1676360171,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,26.317329,depositCollateral,Silo


In [623]:
allOhmBorrowMarketEvents = pd.DataFrame()
allOhmBorrowMarketEvents = pd.concat([ohmDeposited,allCollateralWithdrawals,allOhmBorrowEvents])
allOhmBorrowMarketEvents['dateTime'] =(pd.to_datetime(allOhmBorrowMarketEvents['transaction_timestamp'],unit='s'))
allOhmBorrowMarketEvents.head(1000)

,transaction_hash,transaction_timestamp,transaction_from,transaction_asset_symbol,transactionReadableAmount,transaction_amountUSD,action,protocol,dateTime
0,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,1676615723,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,4.973204,52.678831,depositCollateral,Euler,2023-02-17 06:35:23
1,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,1676379695,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,30000.000000,310089.501811,depositCollateral,Euler,2023-02-14 13:01:35
0,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,1676611691,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,10.000000,105.925340,depositCollateral,Silo,2023-02-17 05:28:11
1,0xf9bbcc923182fb6406e97fce0f92c22c87a284d55812...,1676379599,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,20000.000000,206726.334541,depositCollateral,Silo,2023-02-14 12:59:59
2,0x0aa351eb8ff1ca2bd6c4e65229f83c7e5b596d72c889...,1676360171,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,26.317329,depositCollateral,Silo,2023-02-14 07:36:11
0,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,25.992718,withdrawCollateral,Silo,2023-02-14 13:50:59
1,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,25.992718,withdrawCollateral,Silo,2023-02-14 13:50:59
2,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,25.992718,withdrawCollateral,Silo,2023-02-14 13:50:59
0,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,1676636519,0xcb6e1613029d790c00f89296808f278d6dc25b2f,OHM,58.000000,615.628222,borrowAsset,Silo,2023-02-17 12:21:59
1,0x6726ea9fa581ab0a5fa26b8a112c95e690500f446c39...,1676391467,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,OHM,1480.000000,15782.938441,borrowAsset,Silo,2023-02-14 16:17:47


In [624]:
path =file+'/allOhmBorrowMarketEvents.csv'
allOhmBorrowMarketEvents.to_csv(path, index = False)

path =file+'/allOhmBorrowMarketEvents_piped.csv'
allOhmBorrowMarketEvents.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(ohmDeposits),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\ohmBorrowing/data/allOhmBorrowMarketEvents_piped.csv file saved, 5  records


In [625]:
ohmBorrowMarketLevels = pd.DataFrame({
    'protocol':['Silo','Euler'],
    'collateralBalance':[collateralBalance,totalDepositedEuler],
    'totalBorrowedOhm':[totalBorrowedOHM,0]
})
ohmBorrowMarketLevels.head(10)

,protocol,collateralBalance,totalBorrowedOhm
0,Silo,20010.000000,1713.320428
1,Euler,30004.973204,0.000000


In [626]:
path =file+'/ohmBorrowMarketLevels.csv'
ohmBorrowMarketLevels.to_csv(path, index = False)

path =file+'/ohmBorrowMarketLevels_piped.csv'
ohmBorrowMarketLevels.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(ohmBorrowMarketLevels),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\ohmBorrowing/data/ohmBorrowMarketLevels_piped.csv file saved, 2  records
